In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import logging
import os
from typing import List, Tuple

import pandas as pd

import dataflow.core as dtfcore
import helpers.hdbg as hdbg
import helpers.henv as henv
import helpers.hprint as hprint
import reconciliation as reconcil

/app/amp/core/signal_processing/decorrelation.py:12: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
hdbg.init_logger(verbosity=logging.INFO)

_LOG = logging.getLogger(__name__)

_LOG.info("%s", henv.get_system_signature()[0])

hprint.config_notebook()

INFO: > cmd='/venv/lib/python3.8/site-packages/ipykernel_launcher.py -f /home/.local/share/jupyter/runtime/kernel-5167d9d8-101d-48ec-829f-314496cfc4c4.json'
-----------------------------------------------------------------------------
This code is not in sync with the container:
code_version='1.4.2' != container_version='1.4.0'
-----------------------------------------------------------------------------
You need to:
- merge origin/master into your branch with `invoke git_merge_master`
- pull the latest container with `invoke docker_pull`
INFO  # Git
  branch_name='CmTask4299_Generalize_DAG_self_consistency_for_all_models'
  hash='f23a6bb23'
  # Last commits:
    * f23a6bb23 Nina Lee checkpoint                                                        (   2 hours ago) Wed May 3 15:41:15 2023  (HEAD -> CmTask4299_Generalize_DAG_self_consistency_for_all_models, origin/CmTask4299_Generalize_DAG_self_consistency_for_all_models)
    * 2a368f437 Nina Lee checkpoint                              

# Build config

In [4]:
# Specify the config directly when running the notebook manually.
# Below is just an example.
dst_root_dir = "/shared_data/ecs/preprod/prod_reconciliation/"
dag_builder_name = "C1b"
start_timestamp_as_str = "20230501_131000"
end_timestamp_as_str = "20230502_130500"
run_mode = "paper_trading"
mode = "scheduled"
config_list = reconcil.build_reconciliation_configs(
    dst_root_dir,
    dag_builder_name,
    start_timestamp_as_str,
    end_timestamp_as_str,
    run_mode,
    mode,
)
config = config_list[0]
print(config)

INFO  run_date='20230501'
INFO  Using run_date=20230501
INFO  dst_root_dir='/shared_data/ecs/preprod/prod_reconciliation/', dag_builder_name='C1b', start_timestamp_as_str='20230501_131000', run_mode='paper_trading'
INFO  run_date='20230501'
INFO  target_dir='/shared_data/ecs/preprod/prod_reconciliation/C1b/20230501/paper_trading'
INFO  system_log_dir='system_log_dir.scheduled.20230501_131000.20230502_130500'
meta: 
  date_str: 20230501
  asset_class: crypto
  run_tca: False
  bar_duration: 5T
system_log_path: 
  prod: /shared_data/ecs/preprod/prod_reconciliation/C1b/20230501/paper_trading/prod/system_log_dir.scheduled.20230501_131000.20230502_130500
  sim: /shared_data/ecs/preprod/prod_reconciliation/C1b/20230501/paper_trading/simulation/system_log_dir
system_config_func_as_str: dataflow_orange.system.Cx.get_Cx_system_config_template_instance('C1b')
research_forecast_evaluator_from_prices: 
  init: 
    price_col: vwap
    prediction_col: vwap.ret_0.vol_adj.shift_-2_hat
    volatility_

# DAG self-consistency check

In [5]:
prod_system_log_dir = config["system_log_path"]["prod"]
prod_dag_path = os.path.join(prod_system_log_dir, "dag/node_io/node_io.data")
# Get DAG node names.
dag_node_names = dtfcore.get_dag_node_names(prod_dag_path)
_LOG.info(
    "First node='%s' / Last node='%s'", dag_node_names[0], dag_node_names[-1]
)

INFO  First node='predict.0.read_data' / Last node='predict.8.process_forecasts'


In [6]:
# Get timestamps for the last DAG node.
dag_node_timestamps = dtfcore.get_dag_node_timestamps(
    prod_dag_path, dag_node_names[-1], as_timestamp=True
)
_LOG.info(
    "First timestamp='%s'/ Last timestamp='%s'",
    dag_node_timestamps[0][0],
    dag_node_timestamps[-1][0],
)

INFO  First timestamp='2023-05-01 09:10:00-04:00'/ Last timestamp='2023-05-02 09:05:00-04:00'


## Check DAG self-consistency.

In [7]:
def load_dag_outputs(
    node_name: str, timestamps: List[pd.Timestamp]
) -> Tuple[pd.DataFrame, pd.DataFrame]:
    # Load DAG output at `bar_timestamp = t`.
    current_df = dtfcore.get_dag_node_output(
        prod_dag_path, node_name, timestamps[1]
    )
    current_df = current_df.sort_index()
    # Load DAG output at `bar_timestamp = t-1`.
    previous_df = dtfcore.get_dag_node_output(
        prod_dag_path, node_name, timestamps[0]
    )
    previous_df = previous_df.sort_index()
    return previous_df, current_df

## Use current and previous timestamp, i.e.  `bar_timestamp = t` and  `bar_timestamp = t-1`

In [8]:
def check_diff(
    previous_df: pd.DataFrame, current_df: pd.DataFrame
) -> pd.DatetimeIndex:
    # Remove the first row from the previous_df and the last one from
    # the current_df to align indices.
    previous_df = previous_df[1:]
    current_df = current_df[:-1]
    compare = previous_df.compare(current_df)
    return compare

### Use the 2 last timestamps.

In [9]:
node_name = dag_node_names[-1]
timestamps = [dag_node_timestamps[-2][0], dag_node_timestamps[-1][0]]
previous_df, current_df = load_dag_outputs(node_name, timestamps)

In [11]:
check_diff(previous_df, current_df)

vwap.ret_0.vol_adj.shift_-2_hat                                                                                                                                                                                                                                                                                                                                                                                                                               vwap.ret_0.vol_adj.c.lag0                                                                                                                                                                                                                                                                                                                                                                                                                               vwap.ret_0.vol_adj.c.lag1                                                           \
                                               1030828978       1464553467       1467591036       1528092593       1776791608       1891737434       1966583502       2099673105       2237530510       2384892553       2425308589       2476706208       2484635488       2540896331       2601760471       2683705052       3065029174       3401245610       4516629366       5115052901       5118394986       6051632686       8717633868       8968126878       9872743573                      1030828978       1464553467       1467591036       1528092593       1776791608       1891737434       1966583502       2099673105       2237530510       2384892553       2425308589       2476706208       2484635488       2540896331       2601760471       2683705052       3065029174       3401245610       4516629366       5115052901       5118394986       6051632686       8717633868       8968126878       9872743573                      1030828978       1464553467       1467591036       1528092593         
                                                     self other       self other       self other       self other       self other       self other       self other       self other       self other       self other       self other       self other       self other       self other       self other       self other       self other       self other       self other       self other       self other       self other       self other       self other       self other                      self other       self other       self other       self other       self other       self other       self other       self other       self other       self other       self other       self other       self other       self other       self other       self other       self other       self other       self other       self other       self other       self other       self other       self other       self other                      self other       self other       self other       self other   
end_timestamp                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        
2023-04-28 09:10:00-04:00         

### Use the 2 first timestamps.

In [12]:
node_name = dag_node_names[-1]
timestamps = [dag_node_timestamps[0][0], dag_node_timestamps[1][0]]
previous_df, current_df = load_dag_outputs(node_name, timestamps)

In [13]:
check_diff(previous_df, current_df)

vwap.ret_0.vol_adj.shift_-2_hat                                                                                                                                                                                                                                                                                                                                                                                                                               vwap.ret_0.vol_adj.c.lag0                                                                                                                                                                                                                                                                                                                                                                                                                               vwap.ret_0.vol_adj.c.lag1                                                           \
                                               1030828978       1464553467       1467591036       1528092593       1776791608       1891737434       1966583502       2099673105       2237530510       2384892553       2425308589       2476706208       2484635488       2540896331       2601760471       2683705052       3065029174       3401245610       4516629366       5115052901       5118394986       6051632686       8717633868       8968126878       9872743573                      1030828978       1464553467       1467591036       1528092593       1776791608       1891737434       1966583502       2099673105       2237530510       2384892553       2425308589       2476706208       2484635488       2540896331       2601760471       2683705052       3065029174       3401245610       4516629366       5115052901       5118394986       6051632686       8717633868       8968126878       9872743573                      1030828978       1464553467       1467591036       1528092593         
                                                     self other       self other       self other       self other       self other       self other       self other       self other       self other       self other       self other       self other       self other       self other       self other       self other       self other       self other       self other       self other       self other       self other       self other       self other       self other                      self other       self other       self other       self other       self other       self other       self other       self other       self other       self other       self other       self other       self other       self other       self other       self other       self other       self other       self other       self other       self other       self other       self other       self other       self other                      self other       self other       self other       self other   
end_timestamp                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        
2023-04-27 09:20:00-04:00         

<b> Every difference output has an interval 25 mins, i.e. from 6:20:00 - 6:45:00. Firstly we have 10 mins between timestamps, i.e. 6:20:00 - 6:30:00, the rest of them is 5 mins intervals. The interval varies for each timestamp. These differences are in the history look-back period. The reason why difference appeared is that `previous_df` has a value, e.g. `-1.3` in the first column, meanwhile `current_df` has `NaN` and so among all columns. This is burn-in interval which we remove in `_prepare_dfs_for_comparison()`. Seems like all comparisons will have 6 rows difference approximately.</b>